In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn import decomposition
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import MinMaxScaler
from sklearn import ensemble, tree, linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.metrics.pairwise import euclidean_distances
import random
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from random import shuffle
import time

In [6]:
pip install git+https://github.com/samueljackson92/coranking.git


  Cloning https://github.com/samueljackson92/coranking.git to /tmp/pip-req-build-_iau7jhk
  Running command git clone -q https://github.com/samueljackson92/coranking.git /tmp/pip-req-build-_iau7jhk
  Created wheel for coranking: filename=coranking-0.1.1-cp36-cp36m-linux_x86_64.whl size=87031 sha256=be675634e27a23616b788ced42664bc8b3712c8971984ba81bc0d33124edaaa4
  Stored in directory: /tmp/pip-ephem-wheel-cache-1bdqh_zn/wheels/8e/e0/11/4ae911b6fc472b8bf6087e29d861d2b798c119603adf4b93fa
Successfully built coranking


In [0]:
import coranking
from coranking.metrics import trustworthiness, continuity, LCMC


In [8]:
link1='https://drive.google.com/open?id=1K2_KnRWrKl0IfCUmKLkFerjITQt8B4lW'

fluff, id = link1.split('=')
print (id) # Verify that you have everything after '='

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train.csv') 
train_df  = pd.read_csv('train.csv')

1K2_KnRWrKl0IfCUmKLkFerjITQt8B4lW


In [0]:
concat_df =train_df.copy()

#columns_to_drop = ['PoolQC','MiscFeature','Alley','Fence','FireplaceQu','GarageCars','Id','SalePrice']
columns_to_drop = ['PoolQC','MiscFeature','Alley','Fence','FireplaceQu','GarageCars','Id','SalePrice','MasVnrArea', 'GarageYrBlt', 'LotFrontage']
concat_df.drop(columns_to_drop, axis=1, inplace=True)

nan_columns=concat_df.columns[concat_df.isna().any()].tolist()
numerical_cols = [f for f in concat_df.columns if concat_df.dtypes[f] != 'object']
containNumNans = list(set(nan_columns) & set(numerical_cols))

scaler = MinMaxScaler()
scaled_values = scaler.fit_transform(concat_df[numerical_cols])
concat_df.loc[:,numerical_cols] = scaled_values

concat_df = pd.get_dummies(concat_df)

In [10]:
labels = train_df['SalePrice']
X_train, X_test, y_train, y_test = train_test_split(concat_df, labels, test_size=0.25, random_state=42)

GBest = ensemble.GradientBoostingRegressor(n_estimators=7000, learning_rate=0.01, max_depth=10, max_features='sqrt',min_samples_leaf=5, min_samples_split=15, loss='ls')
GB_model = GBest.fit(X_train, y_train)
preds = GBest.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))


RMSE: 24703.136491


###PCA

In [0]:
from sklearn.decomposition import PCA

from prettytable import PrettyTable
t_mds_10k = PrettyTable(['RMSE_onISOMAPData', 'Dimensions', 'ISOMAP_time','GBR_time' ,'total_time', 'trust_ISOMAP','cont_ISOMAP' ,'lcmc_ISOMAP'])


def MDS_Decision_Tree (training_data, components,size):

  #Reducing to mentioned components
  #print("starting mds")
  mds_start_time = time.time()
  pca = PCA(components).fit(training_data)
  training_data_mds = pca.transform(training_data)
  mds_time = time.time() - mds_start_time
  #print("ending mds")
  
  #Applying Decision Tree
  X_train, X_test, y_train, y_test = train_test_split(training_data_mds, labels[:size], test_size=0.25, random_state=49)
  clf_start_time = time.time()
  GBest = ensemble.GradientBoostingRegressor(n_estimators=7000, learning_rate=0.01, max_depth=10, max_features='sqrt',min_samples_leaf=5, min_samples_split=15, loss='ls')
  GB_model = GBest.fit(X_train, y_train)
  preds = GBest.predict(X_test)
  rmse = np.sqrt(mean_squared_error(y_test, preds))
  clf_time = time.time() - clf_start_time
  score=rmse
  score = str(round(score, 4))

  total_time = mds_time+clf_time
  mds_time = str(round(mds_time, 4))
  clf_time = str(round(clf_time, 4))
  total_time = str(round(total_time, 4))


  trust_mds = 0
  cont_mds = 0
  lcmc_mds = 0
  
  Q = coranking.coranking_matrix(training_data[:size], training_data_mds)
  trust_mds = trustworthiness(Q, min_k=1, max_k=50).sum()/50
  cont_mds = continuity(Q, min_k=1, max_k=50).sum()/50
  lcmc_mds = LCMC(Q, min_k=1, max_k=50).sum()/50

  trust_mds = str(round(trust_mds, 4))
  cont_mds = str(round(cont_mds, 4))
  lcmc_mds = str(round(lcmc_mds, 4))
  
  #print(components)
  print("{}, {}, {}, {}, {}, {}, {}, {} ".format( score, components, mds_time, clf_time ,total_time, trust_mds,cont_mds ,lcmc_mds))
  print("*"*40)
  t_mds_10k.add_row([score, components, mds_time, clf_time ,total_time, trust_mds,cont_mds ,lcmc_mds])
  return

In [23]:
dimensions=266
size=1460
while dimensions >= 2:
  MDS_Decision_Tree (concat_df, dimensions,size)
  dimensions=int(dimensions/2)

40202.7133, 266, 0.1049, 37.7186, 37.8234, 0.98, 0.98, 0.9639 
****************************************
38187.8505, 133, 0.2966, 27.9082, 28.2048, 0.9798, 0.9799, 0.9208 
****************************************
36822.5342, 66, 0.1317, 20.8481, 20.9798, 0.978, 0.9791, 0.7892 
****************************************
38402.8965, 33, 0.0482, 14.8211, 14.8693, 0.9706, 0.9753, 0.627 
****************************************
39965.1019, 16, 0.0447, 12.0645, 12.1091, 0.9464, 0.963, 0.4569 
****************************************
40873.2233, 8, 0.0413, 7.3456, 7.3869, 0.9129, 0.9451, 0.3456 
****************************************
50980.7654, 4, 0.0398, 6.7339, 6.7737, 0.8582, 0.9184, 0.2307 
****************************************
62836.9825, 2, 0.0484, 4.3699, 4.4182, 0.7797, 0.8804, 0.117 
****************************************


In [24]:
print(t_mds_10k)

+-------------------+------------+-------------+----------+------------+--------------+-------------+-------------+
| RMSE_onISOMAPData | Dimensions | ISOMAP_time | GBR_time | total_time | trust_ISOMAP | cont_ISOMAP | lcmc_ISOMAP |
+-------------------+------------+-------------+----------+------------+--------------+-------------+-------------+
|     40202.7133    |    266     |    0.1049   | 37.7186  |  37.8234   |     0.98     |     0.98    |    0.9639   |
|     38187.8505    |    133     |    0.2966   | 27.9082  |  28.2048   |    0.9798    |    0.9799   |    0.9208   |
|     36822.5342    |     66     |    0.1317   | 20.8481  |  20.9798   |    0.978     |    0.9791   |    0.7892   |
|     38402.8965    |     33     |    0.0482   | 14.8211  |  14.8693   |    0.9706    |    0.9753   |    0.627    |
|     39965.1019    |     16     |    0.0447   | 12.0645  |  12.1091   |    0.9464    |    0.963    |    0.4569   |
|     40873.2233    |     8      |    0.0413   |  7.3456  |   7.3869   |